#Tarefa 2a: resumo de texto com arquivos pequenos com o Amazon Nova Lite


Neste caderno, você ingere uma pequena sequência de texto diretamente na API Amazon Bedrock (com o modelo Amazon Nova Lite) e a orienta a resumir o conteúdo fornecido. Você pode aplicar essa abordagem para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, publicações de blogs e outros conteúdos relevantes quando o tamanho do texto de entrada estiver dentro dos limites de tamanho do contexto do modelo.

## Tarefa 2a.1: configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## Tarefa 2a.2: escrever prompt com o texto a ser resumido

Nesta tarefa, você usará um pequeno trecho do texto com menos tokens do que o tamanho máximo permitido pelo modelo de base. Como exemplo de texto de entrada para este laboratório, você usa um parágrafo de uma [postagem do Blog da AWS] (https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) que anuncia o Amazon Bedrock.

O prompt começa com uma instrução `Please provide a summary of the following text.`. 

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs for text and images—including Amazons Titan FMs, which consist of two new LLMs we're also announcing today—through a scalable, reliable, and secure AWS managed service. With Bedrock's serverless experience, customers can easily find the right model for what they're trying to get done, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## Tarefa 2a.3: Criar corpo da solicitação com parâmetros de inferência e prompt 

Nesta tarefa, você criará o corpo do prompt com os parâmetros de inferência e prompt acima.

In [ ]:
# Nova Lite request body
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 2048,
        "temperature": 0,
        "topP": 0.9
    }
})

## Tarefa 2a.4: Invocar o modelo de base via Boto3

Nesta tarefa, você enviará uma solicitação de API para o Amazon Bedrock especificando os parâmetros da solicitação: `modelId`, `accept` e `contentType`. Após o prompt fornecido, o modelo de base no Amazon Bedrock resume o texto de entrada.

### Concluir geração de saída

Por padrão, o serviço Amazon Bedrock gera o resumo completo de um determinado prompt em uma única saída. Pode demorar se a saída do modelo contiver muitos tokens. 

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.nova-lite-v1:0'
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    
    outputText = response_body.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\nTo troubeshoot this issue please refer to the following resources.\nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### Transmitir geração de saída

Em seguida, você verá como usar a API invoke_model_with_response_stream do Amazon Bedrock para transmitir saídas do modelo para que os usuários possam consumi-las conforme são geradas. Em vez de gerar a saída completa de uma só vez, essa API retorna um ResponseStream que envia blocos de saída menores do modelo conforme são produzidos. Você pode exibir essas saídas de transmissão em uma visualização contínua e consumível.

Cada bloco inclui uma carga binária que, ao ser decodificada, revela o fragmento de texto junto com metadados, como a contagem de tokens e o status de conclusão. Cada bloco inclui uma carga binária que, ao ser decodificada, revela o fragmento de texto junto com metadados, como a contagem de tokens e o status de conclusão. Por exemplo, no Nova Lite, cada bloco contém o fragmento de texto, a contagem de tokens e os indicadores de motivo de parada. Esse formato bruto demonstra como o modelo gera conteúdo de forma incremental, palavra por palavra ou frase por frase, em vez de tudo de uma vez. Na célula a seguir, processaremos esses blocos para criar uma exibição mais legível e contínua do conteúdo gerado.

In [ ]:
#invoke model with response stream
modelId = 'amazon.nova-lite-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')

# Collect only the first few chunks
output = []
for i, event in enumerate(stream):
    output.append(event)
    if i >= 4:  # Show only first 5 chunks
        print(f"... and {i} more chunks (truncated)")
        break

# Display truncated output
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.nova-lite-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            if 'contentBlockDelta' in chunk_obj:
                text = chunk_obj['contentBlockDelta']['delta']['text']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
                i+=1

Você testou o uso do SDK boto3 para acessar a API do Amazon Bedrock. Esse SDK fornece acesso programático básico aos recursos do Bedrock. Ao usar essa API, você conseguiu implementar dois casos de uso: 1) Gerar um resumo de texto completo do conteúdo de notícias da AWS de uma só vez e 2) Transmitir a saída resumida em blocos para processamento incremental.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com **Task2b.ipynb**.